### Test Notebook for running Cortex AI with local code

To run this notebook, 

1. Make sure to have Python 3.12.4 installed. That is the latest version with a wheel as seen in this post (https://stackoverflow.com/questions/79135647/pip-install-snowflake-connector-python-fails-building-wheels). Using later versions will demand an installation of Visual Studio C++ 14.0, which i didn't test.

2. create an environment and install the requirements.txt file

3. Create Snowflake credentials in windows credential manager (Follow this post: https://medium.com/@aarhar/password-management-in-python-keyring-and-credential-manager-29fa4ccc919e)

4. Run a query to test you connection (Currently it's not working, but the "basics" are there.)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keyring
import os 
import snowflake.connector as sf_connector

In [3]:
account_identifier = keyring.get_password('NC_Snowflake_Trial_Account_Name', 'account_identifier')
user_name = "EMILHALDAN5468402"
password = keyring.get_password('NC_Snowflake_Trial_User_Password', user_name)

print("Account Identifier: ", account_identifier)
print("User Name: ", user_name)
# print("Password: ", password)


Account Identifier:  EDBHJWL-MFB05236
User Name:  EMILHALDAN5468402


## Setup in snowflake 

1. Create the database structure in Python first, and then create database tables in SnowFlake once the processing is done. This is due to the chaotic nature of the PDF files, which require a lot of cleaning and processing.

2. 


## The intended database structure is as follows:

- **manuals** (Stores metadata about each manual)  
  - `manual_id` (Unique ID for each manual)  
  - `title` (Title of the manual)  
  - `version` (Version or revision number)  
  - `language` (Language code, e.g., 'en', 'de')  
  - `source_path` (Original PDF file path or S3 URL)  
  - `created_at` (Timestamp when the manual was added)  

- **sections** (Defines logical sections and subsections within each manual)  
  - `section_id` (Unique ID for the section)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `title` (Title or heading of the section)  
  - `order_num` (Numerical order of the section in the manual)  
  - `parent_section_id` (Optional FK for nested subsections)  

- **chunks** (Text chunks derived from section content for LLMs or search)  
  - `chunk_id` (Unique ID for the chunk)  
  - `section_id` (Foreign key referencing `sections`)  
  - `chunk_text` (The text content of the chunk)  
  - `chunk_order` (Order of the chunk within the section)  
  - `embedding` (Optional vector for semantic search or embeddings)  

- **images** (Stores references to images extracted from the manual)  
  - `image_id` (Unique ID for the image)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `section_id` (Foreign key referencing `sections`)  
  - `image_path` (S3 or web-accessible path to the image)  
  - `caption` (Optional caption or alt text)  
  - `order_num` (Display order within the section)  


In [6]:
manuals_df = pd.DataFrame(columns=['Manual_ID', 'Doc_Name', 'Version', 'Language', 'Source_Path'])
sections_df = pd.DataFrame(columns=['Section_ID', 'Manual_ID', 'Doc_Name', 'Order_Num', 'Parent_Section_ID'])
chunks_df = pd.DataFrame(columns=['Chunk_ID', 'Section_ID', 'Manual_ID', 'Order_Num', 'Chunk_Embedding_Vector'])
images_df = pd.DataFrame(columns=['Image_ID', 'Section_ID', 'Manual_ID', 'Order_Num', 'Image_Description' ,'Image_Path'])

In [5]:
pdf_files_path = "./Washer_Manuals"

for filename in os.listdir(pdf_files_path):
    if filename.endswith(".pdf"):
        print(filename)
        # do something with the file
    else:
        continue

k714wm14 service manual.pdf
mmo_87050793_1630397705_64_10689.pdf
technical-manual-w11663204-revb.pdf
WAK20160IN.pdf
WAN28258GB.pdf
WAN28282GC.pdf
Washing machine Top-loader C series.pdf
WAT24168IN.pdf
WAV28KH3GB.pdf
WFL2050.pdf
WGA1340SIN.pdf
WGA1420SIN.pdf
WGE03408GB.pdf
WGG254Z0GB.pdf


In [ ]:

connection_parameters = {
    "account_identifier": account_identifier,
    "user": user_name,
    "password": password,
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "CYBERSYN",
    "schema": "PUBLIC"
}

# Connect to Snowflake
conn = sf_connector.connect(
    user=connection_parameters['user'],
    password=connection_parameters['password'],
    account=connection_parameters['account_identifier'],
    warehouse=connection_parameters['warehouse'],
    database=connection_parameters['database'],
    schema=connection_parameters['schema'],
    role=connection_parameters['role']
)

# # Test the connection
cursor = conn.cursor()
cursor.execute("""
    SELECT * 
    FROM company_metadata;
""")

df = cursor.fetch_pandas_all()
df